# 地理信息系统设计与开发（multipolygon相关处理）

In [1]:
import os
import ogr
import osr
import json
import requests
import matplotlib.pyplot as plt
%matplotlib inline

## 显示multipolygon

In [2]:
def myDisplayMultiPolygon(myMultipolygon):
    for i in range(myMultipolygon.GetGeometryCount()):
        ring=myMultipolygon.GetGeometryRef(i)
        #print('ring:',ring)
        subring=ring.GetGeometryRef(0) 
        #print("subring:",subring)   
        coords=subring.GetPoints()
        #print("coords:",coords)
        x,y=zip(*coords)
        #print("x=",x,"y=",y)
        plt.plot(x,y,'black')
        plt.fill(x,y,'p')

## 自适应显示polygon和multipolygon

In [3]:
def myDisplayPolygon(lyr):
    for row in lyr:
        print("row:",row)
        geom=row.geometry()
        print('geom:',geom.GetGeometryName(), 'count:',geom.GetGeometryCount())
        print("geom=",geom)

        if(geom.GetGeometryName() == 'MULTIPOLYGON'):
            for i in range(geom.GetGeometryCount()):
                ring=geom.GetGeometryRef(i)
                print('ring:',ring)
                subring=ring.GetGeometryRef(0) 
                print("subring:",subring)   
                coords=subring.GetPoints()
                print("coords:",coords)
                x,y=zip(*coords)
                #print("x=",x,"y=",y)
                plt.plot(x,y,'black')
                plt.fill(x,y,'p')
        else:
            ring=geom.GetGeometryRef(0)
            print("ring:",ring)
            coords=ring.GetPoints()
            print("coords:",coords)
            x,y=zip(*coords)
        #print("x=",x,"y=",y)
        plt.plot(x,y,'black')
        plt.fill(x,y,'p')

In [4]:
def getChinaGeojson():
    ds=ogr.Open('china.json')
    lyr=ds.GetLayer()
    print(lyr.GetFeatureCount())
    myDisplayPolygon(lyr)

## 由Geojson创建Multipolygon

In [6]:
def createMultiPolygonFromPloygonJson():
    myMultipolygon = ogr.Geometry(ogr.wkbMultiPolygon)
    ds=ogr.Open('yunnan.json')
    lyr=ds.GetLayer()
    for feature in lyr:
        geom=feature.geometry()
        myMultipolygon.AddGeometry(geom)
    print(myMultipolygon.ExportToWkt())
    return myMultipolygon

## 由Geojson创建shp

In [ ]:
def createPolygonShpFromPloygonJson():
    shpdriver = ogr.GetDriverByName('ESRI Shapefile')
    outputfn='yunnan.shp'
    if os.path.exists(outputfn):
        shpdriver.DeleteDataSource(outputfn)
    outputds = shpdriver.CreateDataSource(outputfn)
    targetSR = osr.SpatialReference()
    targetSR.ImportFromEPSG(4326) #Geo WGS84
    outlyr = outputds.CreateLayer(outputfn,targetSR,geom_type=ogr.wkbPolygon)
    featureDefn = outlyr.GetLayerDefn()
    ds=ogr.Open('yunnan.json')
    lyr=ds.GetLayer()
    for feature in lyr:
        geom=feature.geometry()
        #print(geom)
        outFeature = ogr.Feature(featureDefn)
        outFeature.SetGeometry(geom)
        #print("outFeature:",outFeature.geometry())
        outlyr.CreateFeature(outFeature)
        outFeature = None 

    ds=None    
    outputds = None   

## 由Geojson自适应创建Multipolygon

In [7]:
def createMultiPolygonFromMultiPolygonJson():
    myMultipolygon = ogr.Geometry(ogr.wkbMultiPolygon)
    ds=ogr.Open('china.json')
    lyr=ds.GetLayer() 
    for feature in lyr:
        geom=feature.geometry()
        if(geom.GetGeometryName() == 'MULTIPOLYGON'):
            for i in range(geom.GetGeometryCount()):
                ring=geom.GetGeometryRef(i)
                #print('ring:',ring)
                myMultipolygon.AddGeometry(ring)
        else:
            myMultipolygon.AddGeometry(geom)
    return myMultipolygon 

In [8]:
def spatialFilterTest(inputfn):
    inputds = ogr.Open(inputfn)
    inputlyr = inputds.GetLayer()
    myMultiPolygon=createMultiPolygonFromMultiPolygonJson()
    print(myMultiPolygon.GetGeometryCount())
    print("before filter:",inputlyr.GetFeatureCount())
    inputlyr.SetSpatialFilter(myMultiPolygon)
    print("after filter:",inputlyr.GetFeatureCount())
    myDisplayMultiPolygon(myMultiPolygon)

    for feature in inputlyr:
        geom=feature.geometry()
        x=geom.GetX()
        y=geom.GetY()
        plt.plot(x,y,'o',markersize=5,color='k')
        plt.text(x,y,feature.GetField('magnitude'))

    return inputlyr

In [9]:
def spatialFilter(inputlyr):
    myMultiPolygon=createMultiPolygonFromMultiPolygonJson()
    print(myMultiPolygon.GetGeometryCount())
    print("before filter:",inputlyr.GetFeatureCount())
    inputlyr.SetSpatialFilter(myMultiPolygon)
    print("after filter:",inputlyr.GetFeatureCount())

    return inputlyr

In [10]:
def createMultipolygonShp():
    myMultiPolygon=createMultiPolygonFromMultiPolygonJson()
    shpdriver = ogr.GetDriverByName('ESRI Shapefile')
    outputfn='china.shp'
    if os.path.exists(outputfn):
        shpdriver.DeleteDataSource(outputfn)
    outputds = shpdriver.CreateDataSource(outputfn)
    targetSR = osr.SpatialReference()
    targetSR.ImportFromEPSG(4326) #Geo WGS84
    lyr = outputds.CreateLayer(outputfn,targetSR,geom_type=ogr.wkbMultiPolygon)
    featureDefn = lyr.GetLayerDefn()

    outFeature = ogr.Feature(featureDefn)
    outFeature.SetGeometry(myMultiPolygon)
    lyr.CreateFeature(outFeature)
    outFeature = None 
    outputds = None 

In [11]:
def reprojectShpFile(filename):
    #不能对multipolygon直接投影，只能对polygon进行投影，
    #multipolygon只能拆解为polygon后投影才会正确
    driver = ogr.GetDriverByName('ESRI Shapefile')

    # input SpatialReference
    inSpatialRef = osr.SpatialReference()
    inSpatialRef.ImportFromEPSG(4326)

    # output SpatialReference
    outSpatialRef = osr.SpatialReference()
    outSpatialRef.ImportFromEPSG(32648)

    # create the CoordinateTransformation
    coordTrans = osr.CoordinateTransformation(inSpatialRef, outSpatialRef)

    # get the input layer
    inDataSet = driver.Open(filename)
    inLayer = inDataSet.GetLayer()

    # create the output layer
    outputShapefile = 'china_32648.shp'
    if os.path.exists(outputShapefile):
        driver.DeleteDataSource(outputShapefile)
    outDataSet = driver.CreateDataSource(outputShapefile)
    outLayer = outDataSet.CreateLayer("china_32648", outSpatialRef, geom_type=ogr.wkbMultiPolygon)

    # add fields
    inLayerDefn = inLayer.GetLayerDefn()
    for i in range(0, inLayerDefn.GetFieldCount()):
        fieldDefn = inLayerDefn.GetFieldDefn(i)
        outLayer.CreateField(fieldDefn)

    # get the output layer's feature definition
    outLayerDefn = outLayer.GetLayerDefn()

    # loop through the input features
    inFeature = inLayer.GetNextFeature()
    while inFeature:
        # get the input geometry
        geom = inFeature.GetGeometryRef()
        print("geom old:",geom)
        spatialRef = geom.GetSpatialReference()
        print(spatialRef)
        # reproject the geometry
        geom.Transform(coordTrans)
        print("geom new:",geom)
        spatialRef = geom.GetSpatialReference()
        print(spatialRef)

        # create a new feature
        outFeature = ogr.Feature(outLayerDefn)
        # set the geometry and attribute
        outFeature.SetGeometry(geom)

        #outFeature.GetGeometryRef().Transform(coordTrans)

        for i in range(0, outLayerDefn.GetFieldCount()):
            outFeature.SetField(outLayerDefn.GetFieldDefn(i).GetNameRef(), inFeature.GetField(i))
        # add the feature to the shapefile
        outLayer.CreateFeature(outFeature)
        # dereference the features and get the next input feature
        outFeature = None
        inFeature = inLayer.GetNextFeature()

    # Save and close the shapefiles
    inDataSet = None
    outDataSet = None

    # outSpatialRef.MorphToESRI()
    # file = open('china_2927.prj', 'w')
    # file.write(outSpatialRef.ExportToWkt())
    # file.close()

In [12]:
def createBuffer(inputfn, outputBufferfn, bufferDist):
    inputds = ogr.Open(inputfn)
    inputlyr = inputds.GetLayer()

    shpdriver = ogr.GetDriverByName('ESRI Shapefile')
    if os.path.exists(outputBufferfn):
        shpdriver.DeleteDataSource(outputBufferfn)
    outputBufferds = shpdriver.CreateDataSource(outputBufferfn)
    targetSR = osr.SpatialReference()
    targetSR.ImportFromEPSG(4326) #Geo WGS84
    bufferlyr = outputBufferds.CreateLayer(outputBufferfn,targetSR,geom_type=ogr.wkbPolygon)
    featureDefn = bufferlyr.GetLayerDefn()

    # result = inputds.ExecuteSQL("select * from test3 where magnitude >= 6")
    # inputlyr.SetAttributeFilter("magnitude >= 6")
    # result=inputlyr
    result = spatialFilter(inputlyr)

    resultFeat = result.GetNextFeature()
    while resultFeat :
       print(resultFeat.GetField('place')) 
       resultFeat = result.GetNextFeature()

    for feature in inputlyr:
        ingeom = feature.GetGeometryRef()
        geomBuffer = ingeom.Buffer(bufferDist)

        outFeature = ogr.Feature(featureDefn)
        outFeature.SetGeometry(geomBuffer)
        bufferlyr.CreateFeature(outFeature)
        outFeature = None